## Import library

In [1]:
import numpy as np
import pandas as pd

from tensorflow.keras.layers import LSTM, Dense,Dropout
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import os

from sklearn.model_selection import train_test_split

## Read dataset

In [2]:
# Đọc dữ liệu
standing_df = pd.read_csv("Standing_kict.text")
eating_df = pd.read_csv("Eating_kict.text")
nothing_df = pd.read_csv("Do_nothing_kict.text")
# clapp_df = pd.read_csv("Clapping.txt")
handswing_df = pd.read_csv("Swing_hand_kict.text")

X = []
y = []
no_of_timesteps = 10

### data processing 

In [3]:
dataset = standing_df.iloc[:,1:].values
n_sample = len(dataset)
for i in range(no_of_timesteps, n_sample):
    X.append(dataset[i-no_of_timesteps:i,:])
    y.append(np.array([1., 0., 0.,0.]))

dataset = eating_df.iloc[:,1:].values
n_sample = len(dataset)
for i in range(no_of_timesteps, n_sample):
    X.append(dataset[i-no_of_timesteps:i,:])
    y.append(np.array([0., 1., 0.,0.]))

dataset = nothing_df.iloc[:,1:].values
n_sample = len(dataset)
for i in range(no_of_timesteps, n_sample):
    X.append(dataset[i-no_of_timesteps:i,:])
    y.append(np.array([0., 0., 1.,0.]))


# dataset = clapp_df.iloc[:,1:].values
# n_sample = len(dataset)
# for i in range(no_of_timesteps, n_sample):
#     X.append(dataset[i-no_of_timesteps:i,:])
#     y.append(np.array([0., 0., 0.,1.,0.]))

dataset = handswing_df.iloc[:,1:].values
n_sample = len(dataset)
for i in range(no_of_timesteps, n_sample):
    X.append(dataset[i-no_of_timesteps:i,:])
    y.append(np.array([0., 0.,0.,1.]))


In [4]:
X, y = np.array(X), np.array(y)
print(X.shape, y.shape)
# print(X)
# columnTransformer = ColumnTransformer([('encoder',
#                                         OneHotEncoder(),
#                                         [0])],
#                                       remainder='passthrough')
# onehotencoder = OneHotEncoder()
# y = onehotencoder.fit_transform(y)
#

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5)
print(X_train.shape)
print(y_train.shape)

(1960, 10, 64) (1960, 4)
(1372, 10, 64)
(1372, 4)


## Training model

0. LSTM

In [6]:
model  = Sequential()
model.add(LSTM(units = 32, input_shape = (X.shape[1], X.shape[2])))
model.add(Dense(units = 4, activation="softmax"))
model.compile(optimizer="adam", metrics = ['accuracy'], loss = "categorical_crossentropy")

model.fit(X_train, y_train, epochs=20, verbose = 1,batch_size=32,validation_data=(X_val, y_val))

Epoch 1/20
43/43 [==============================] - 8s 44ms/step - loss: 1.3328 - accuracy: 0.4652 - val_loss: 0.6537 - val_accuracy: 0.9116
Epoch 2/20
43/43 [==============================] - 0s 9ms/step - loss: 0.4711 - accuracy: 0.9342 - val_loss: 0.2662 - val_accuracy: 0.9320
Epoch 3/20
43/43 [==============================] - 0s 10ms/step - loss: 0.1996 - accuracy: 0.9684 - val_loss: 0.1394 - val_accuracy: 0.9932
Epoch 4/20
43/43 [==============================] - 0s 10ms/step - loss: 0.1012 - accuracy: 0.9902 - val_loss: 0.0644 - val_accuracy: 1.0000
Epoch 5/20
43/43 [==============================] - 0s 9ms/step - loss: 0.0480 - accuracy: 1.0000 - val_loss: 0.0318 - val_accuracy: 1.0000
Epoch 6/20
43/43 [==============================] - 0s 11ms/step - loss: 0.0244 - accuracy: 1.0000 - val_loss: 0.0204 - val_accuracy: 1.0000
Epoch 7/20
43/43 [==============================] - 0s 11ms/step - loss: 0.0168 - accuracy: 1.0000 - val_loss: 0.0142 - val_accuracy: 1.0000
Epoch 8/20
43/4

In [7]:
lstm_prediction = model.predict(X_test)
lstm_prediction_arg = np.argmax(lstm_prediction, axis=1)
y_test_arg=np.argmax(y_test, axis=1)

In [8]:
from sklearn.metrics import classification_report
table = classification_report(y_test_arg, lstm_prediction_arg, output_dict=False,
    target_names=['Standing', 'Eating', 'Nothing','Handswing'] )
print(table)

              precision    recall  f1-score   support

    Standing       1.00      1.00      1.00        65
      Eating       1.00      1.00      1.00        76
     Nothing       1.00      1.00      1.00        71
   Handswing       1.00      1.00      1.00        82

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294



1. Stacked LSTM

In [9]:

from tabnanny import verbose


stlstm_model  = Sequential()
stlstm_model.add(LSTM(units = 50, return_sequences = True, input_shape = (X.shape[1], X.shape[2])))
stlstm_model.add(Dropout(0.2))
stlstm_model.add(LSTM(units = 50, return_sequences = True))
stlstm_model.add(Dropout(0.2))
stlstm_model.add(LSTM(units = 50, return_sequences = True))
stlstm_model.add(Dropout(0.2))
stlstm_model.add(LSTM(units = 50))
stlstm_model.add(Dropout(0.2))
stlstm_model.add(Dense(units =4, activation="softmax"))
stlstm_model.compile(optimizer="adam", metrics = ['accuracy'], loss = "categorical_crossentropy")

stlstm_model.fit(X_train, y_train, epochs=20, verbose = 1,batch_size=32,validation_data=(X_val, y_val))

Epoch 1/20
43/43 [==============================] - 18s 111ms/step - loss: 1.0467 - accuracy: 0.6398 - val_loss: 0.0167 - val_accuracy: 0.9966
Epoch 2/20
43/43 [==============================] - 1s 24ms/step - loss: 0.0592 - accuracy: 0.9879 - val_loss: 0.0104 - val_accuracy: 1.0000
Epoch 3/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0324 - accuracy: 0.9874 - val_loss: 0.0035 - val_accuracy: 1.0000
Epoch 4/20
43/43 [==============================] - 1s 25ms/step - loss: 0.0280 - accuracy: 0.9918 - val_loss: 0.0246 - val_accuracy: 0.9898
Epoch 5/20
43/43 [==============================] - 1s 23ms/step - loss: 0.0378 - accuracy: 0.9924 - val_loss: 0.0035 - val_accuracy: 1.0000
Epoch 6/20
43/43 [==============================] - 1s 22ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.0017 - val_accuracy: 1.0000
Epoch 7/20
43/43 [==============================] - 1s 22ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 8/20


In [10]:
stacked_lstm_prediction = stlstm_model.predict(X_test)
print(stacked_lstm_prediction)

[[5.7708101e-05 4.8345573e-05 9.9984145e-01 5.2477870e-05]
 [8.2495339e-05 9.9976391e-01 9.5470205e-05 5.8193978e-05]
 [5.4824905e-05 4.5146917e-05 9.9984550e-01 5.4445376e-05]
 ...
 [7.8124489e-05 9.9977082e-01 9.6298791e-05 5.4753560e-05]
 [4.4734428e-05 2.2294626e-05 7.0753522e-05 9.9986219e-01]
 [4.4850523e-05 2.1686621e-05 7.0601105e-05 9.9986291e-01]]


In [11]:
stack_lstm_prediction_arg = np.argmax(stacked_lstm_prediction, axis=1)

y_test_arg=np.argmax(y_test, axis=1)

In [12]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
## Accuracy
lstm_acc = accuracy_score(y_test_arg,stack_lstm_prediction_arg)
## Precision
lstm_pre = precision_score(y_test_arg,stack_lstm_prediction_arg,average=None)
## Recall
lstm_rec = recall_score(y_test_arg,stack_lstm_prediction_arg,average=None)
## F1 Score
lstm_f1 = f1_score(y_test_arg,stack_lstm_prediction_arg,average=None)

print("accuracy = ", lstm_acc)
print("precision = ", lstm_pre)
print("recall = ", lstm_rec)
print("f1_score = ", lstm_f1)

accuracy =  1.0
precision =  [1. 1. 1. 1.]
recall =  [1. 1. 1. 1.]
f1_score =  [1. 1. 1. 1.]


In [13]:
from sklearn.metrics import classification_report
table = classification_report(y_test_arg, stack_lstm_prediction_arg, output_dict=False,
    target_names=['Standing', 'Eating', 'Nothing','Handswing'] )
print(table)

              precision    recall  f1-score   support

    Standing       1.00      1.00      1.00        65
      Eating       1.00      1.00      1.00        76
     Nothing       1.00      1.00      1.00        71
   Handswing       1.00      1.00      1.00        82

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294



2. Random Forest

In [14]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
#y_probas_forest = cross_val_predict(forest_clf, X_train, y_train, cv=3, method="predict_proba")
X_train_fr = X_train.reshape(-1, X_train.shape[1] * X_train.shape[2])
y_train_arg = np.argmax(y_train, axis=1)
print(X_train_fr.shape)
forest_clf.fit(X_train_fr, y_train_arg)

(1372, 640)


RandomForestClassifier(random_state=42)

In [15]:
from sklearn.metrics import classification_report
X_test_fr = X_test.reshape(-1, X_test.shape[1] * X_test.shape[2])
rf_prediction_arg = forest_clf.predict(X_test_fr)
y_test_arg=np.argmax(y_test, axis=1)
table = classification_report(y_test_arg, rf_prediction_arg, output_dict=False,
    target_names=['Standing', 'Eating', 'Nothing','Handswing'] )
print(table)

              precision    recall  f1-score   support

    Standing       1.00      1.00      1.00        65
      Eating       1.00      1.00      1.00        76
     Nothing       1.00      1.00      1.00        71
   Handswing       1.00      1.00      1.00        82

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294



3. LogisticRegression

In [16]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(random_state=42)
#y_probas_forest = cross_val_predict(forest_clf, X_train, y_train, cv=3, method="predict_proba")
X_train_fr = X_train.reshape(-1, X_train.shape[1] * X_train.shape[2])
y_train_arg = np.argmax(y_train, axis=1)
lr_clf.fit(X_train_fr, y_train_arg)

C:\Users\Hoang\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [17]:
from sklearn.metrics import classification_report
X_test_fr = X_test.reshape(-1, X_test.shape[1] * X_test.shape[2])
lr_prediction_arg = lr_clf.predict(X_test_fr)
y_test_arg=np.argmax(y_test, axis=1)
table = classification_report(y_test_arg, lr_prediction_arg, output_dict=False,
    target_names=['Standing', 'Eating', 'Nothing','Handswing'] )
print(table)

              precision    recall  f1-score   support

    Standing       1.00      1.00      1.00        65
      Eating       1.00      1.00      1.00        76
     Nothing       1.00      1.00      1.00        71
   Handswing       1.00      1.00      1.00        82

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294



4. SVC

In [18]:
from sklearn.svm import SVC
svm_clf = SVC()
X_train_fr = X_train.reshape(-1, X_train.shape[1] * X_train.shape[2])
y_train_arg = np.argmax(y_train, axis=1)
svm_clf.fit(X_train_fr, y_train_arg)

SVC()

In [19]:
from sklearn.metrics import classification_report
X_test_fr = X_test.reshape(-1, X_test.shape[1] * X_test.shape[2])
svm_prediction_arg = svm_clf.predict(X_test_fr)
y_test_arg=np.argmax(y_test, axis=1)
table = classification_report(y_test_arg, svm_prediction_arg, output_dict=False,
    target_names=['Standing', 'Eating', 'Nothing','Handswing'] )
print(table)

              precision    recall  f1-score   support

    Standing       1.00      1.00      1.00        65
      Eating       1.00      1.00      1.00        76
     Nothing       1.00      1.00      1.00        71
   Handswing       1.00      1.00      1.00        82

    accuracy                           1.00       294
   macro avg       1.00      1.00      1.00       294
weighted avg       1.00      1.00      1.00       294

